##### 本质和 rerank 的训练方式相似，二者都是基于AutoModelForSequenceClassifier ,设置num_label = 1 得到预测值，区别是二者计算loss 的方式不同

In [2]:
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer

dataset = load_dataset('parquet', data_files=['C:/Users/zxd/Desktop/00_手写实现/06_dpo/data/test-00000-of-00001.parquet'])

In [5]:
dataset['train'][0]

{'chosen': [{'content': 'As an HR manager, you want to test a potential employee\'s ability to solve puzzles to determine their suitability for a job. Write a Python script that generates a list of questions that require logical reasoning to answer. Your list should include questions related to mathematical puzzles, language puzzles, logic puzzles, lateral thinking puzzles, and pattern recognition puzzles. Use the following code as a starting point:\nquestions = {\n    "Mathematical puzzles": ["If the value of x+y = 20 and x-y = 10, what is the value of x and y?", "If a pizza has a radius of 8 inches and is cut into 6 equal slices, what is the area of each slice?"],\n    "Language puzzles": ["What word starts with \'e\' and ends with \'e\' but only contains one letter?", "I am taken from a mine, and shut up in a wooden case, from which I am never released, and yet I am used by almost every person. What am I?"],\n    "Logic puzzles": ["You have 3 boxes. One contains only apples, one con

In [8]:
dataset['train'].select(range(20))


Dataset({
    features: ['chosen', 'rejected', 'score_chosen', 'score_rejected'],
    num_rows: 20
})

In [9]:
dataset = load_dataset("./data", split="train")

Generating train split: 62135 examples [00:00, 218458.28 examples/s]
Generating test split: 1000 examples [00:00, ? examples/s]


In [10]:
dataset

Dataset({
    features: ['chosen', 'rejected', 'score_chosen', 'score_rejected'],
    num_rows: 62135
})

In [ ]:
ds = dataset.select(range(3))
ds



Map: 100%|██████████| 3/3 [00:00<00:00, 247.41 examples/s]


Dataset({
    features: ['chosen', 'rejected', 'score_chosen', 'score_rejected', 'prompt'],
    num_rows: 3
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("E:/Model/Qwen2-0.5B-Instruct")

def apply_chat_template(
    example: dict[str, list[dict[str, str]]],
    tokenizer,
    tools = None,
) -> dict[str, str]:
    r"""
    Apply a chat template to a conversational example along with the schema for a list of functions in `tools`.

    For more details, see [`maybe_apply_chat_template`].
    """
    # Check that the example has the correct keys
    supported_keys = ["prompt", "chosen", "rejected", "completion", "messages", "label"]
    example_keys = {key for key in example.keys() if key in supported_keys}
    if example_keys not in [
        {"messages"},  # language modeling
        {"prompt"},  # prompt-only
        {"prompt", "completion"},  # prompt-completion
        {"prompt", "chosen", "rejected"},  # preference
        {"chosen", "rejected"},  # preference with implicit prompt
        {"prompt", "completion", "label"},  # unpaired preference
    ]:
        raise KeyError(f"Invalid keys in the example: {example_keys}")

    # Apply the chat template to the whole conversation
    if "messages" in example:
        messages = tokenizer.apply_chat_template(example["messages"], tools=tools, tokenize=False)

    # Apply the chat template to the prompt, adding the generation prompt
    if "prompt" in example:
        last_role = example["prompt"][-1]["role"]
        if last_role == "user":
            add_generation_prompt = True
            continue_final_message = False
        elif last_role == "assistant":
            add_generation_prompt = False
            continue_final_message = True
        else:
            raise ValueError(f"Invalid role in the last message: {last_role}")
        prompt = tokenizer.apply_chat_template(
            example["prompt"],
            tools=tools,
            continue_final_message=continue_final_message,
            tokenize=False,
            add_generation_prompt=add_generation_prompt,
        )

    # Apply the chat template to the entire prompt + completion
    if "prompt" in example:  # explicit prompt and prompt-completion case
        if "chosen" in example:
            prompt_chosen = tokenizer.apply_chat_template(
                example["prompt"] + example["chosen"], tools=tools, tokenize=False
            )
            chosen = prompt_chosen[len(prompt) :]
        if "rejected" in example and "prompt" in example:  # explicit prompt
            prompt_rejected = tokenizer.apply_chat_template(
                example["prompt"] + example["rejected"], tools=tools, tokenize=False
            )
            rejected = prompt_rejected[len(prompt) :]
        if "completion" in example:
            prompt_completion = tokenizer.apply_chat_template(
                example["prompt"] + example["completion"], tools=tools, tokenize=False
            )
            completion = prompt_completion[len(prompt) :]
    else:  # implicit prompt case
        if "chosen" in example:
            chosen = tokenizer.apply_chat_template(example["chosen"], tools=tools, tokenize=False)
        if "rejected" in example:
            rejected = tokenizer.apply_chat_template(example["rejected"], tools=tools, tokenize=False)

    # Ensure that the prompt is the initial part of the prompt-completion string
    if "prompt" in example:
        error_message = (
            "The chat template applied to the prompt + completion does not start with the chat template applied to "
            "the prompt alone. This can indicate that the chat template is not supported by TRL."
            "\n**Prompt**:\n{}\n\n**Prompt + Completion**:\n{}"
        )
        if "chosen" in example and not prompt_chosen.startswith(prompt):
            raise ValueError(error_message.format(prompt, prompt_chosen))
        if "rejected" in example and not prompt_rejected.startswith(prompt):
            raise ValueError(error_message.format(prompt, prompt_rejected))
        if "completion" in example and not prompt_completion.startswith(prompt):
            raise ValueError(error_message.format(prompt, prompt_completion))

    # Extract the completion by removing the prompt part from the prompt-completion string
    output = {}
    if "messages" in example:
        output["text"] = messages
    if "prompt" in example:
        output["prompt"] = prompt
    if "chosen" in example:
        output["chosen"] = chosen
    if "rejected" in example:
        output["rejected"] = rejected
    if "completion" in example:
        output["completion"] = completion
    if "label" in example:
        output["label"] = example["label"]

    return output


Map: 100%|██████████| 3/3 [00:00<00:00, 170.87 examples/s]


In [39]:
train_dataset = ds.map(apply_chat_template, fn_kwargs={"tokenizer": tokenizer})

In [40]:
train_dataset[0]

{'chosen': "<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nUse the pygame library to write a version of the classic game Snake, with a unique twist<|im_end|>\n<|im_start|>assistant\nSure, I'd be happy to help you write a version of the classic game Snake using the pygame library! Here's a basic outline of how we can approach this:\n\n1. First, we'll need to set up the game display and create a game object that we can use to handle the game's state.\n2. Next, we'll create the game's grid, which will be used to represent the game board. We'll need to define the size of the grid and the spaces within it.\n3. After that, we'll create the snake object, which will be used to represent the player's movement. We'll need to define the size of the snake and the speed at which it moves.\n4. We'll also need to create a food object, which will be used to represent the food that the player must collect to score points. We'll need to define the location of the food and 

In [36]:
from typing import Any 
def tokenize(batch, tokenizer):
    """Tokenize a batch from a reward modelling dataset."""
    new_examples = {
        "input_ids_chosen": [],
        "attention_mask_chosen": [],
        "input_ids_rejected": [],
        "attention_mask_rejected": [],
    }
    for chosen, rejected in zip(batch["chosen"], batch["rejected"]):
        tokenized_chosen = tokenizer(chosen)
        # print(tokenized_chosen)
        tokenized_rejected = tokenizer(rejected)
        new_examples["input_ids_chosen"].append(tokenized_chosen["input_ids"])
        # print(new_examples["input_ids_chosen"])
        new_examples["attention_mask_chosen"].append(tokenized_chosen["attention_mask"])
        new_examples["input_ids_rejected"].append(tokenized_rejected["input_ids"])
        new_examples["attention_mask_rejected"].append(tokenized_rejected["attention_mask"])

    return new_examples

In [41]:
fn_kwargs = {"tokenizer": tokenizer}

train_dataset_ = train_dataset.map(
    tokenize,
    batched=True,
    fn_kwargs=fn_kwargs,
    num_proc=5,
)

num_proc must be <= 3. Reducing num_proc to 3 for dataset of size 3.
Map (num_proc=3): 100%|██████████| 3/3 [00:04<00:00,  1.63s/ examples]


In [42]:
train_dataset_[0].keys()

dict_keys(['chosen', 'rejected', 'score_chosen', 'score_rejected', 'input_ids_chosen', 'attention_mask_chosen', 'input_ids_rejected', 'attention_mask_rejected'])

In [45]:
type(train_dataset_[0]['input_ids_chosen']), len(train_dataset_[0]['input_ids_chosen']) , train_dataset_[0]['input_ids_chosen'][:2]


(list, 889, [151644, 8948])

In [56]:
def add_prompt(example):
    prompt = [{'role': 'user', 'content': '*******************8'}, {'role': 'assistant', 'content': '你好，这是一个添加的 prompt，现在开始测试'}]
    return {'prompt' : prompt}

ds1 = ds.map(add_prompt)
ds1

Map: 100%|██████████| 3/3 [00:00<00:00, 583.98 examples/s]


Dataset({
    features: ['chosen', 'rejected', 'score_chosen', 'score_rejected', 'prompt'],
    num_rows: 3
})

In [57]:
ds1[0]['prompt']

[{'content': '*******************8', 'role': 'user'},
 {'content': '你好，这是一个添加的 prompt，现在开始测试', 'role': 'assistant'}]

In [58]:
train_dataset1 = ds1.map(apply_chat_template, fn_kwargs={"tokenizer": tokenizer})
train_dataset1

Map: 100%|██████████| 3/3 [00:00<00:00, 1002.38 examples/s]


Dataset({
    features: ['chosen', 'rejected', 'score_chosen', 'score_rejected', 'prompt'],
    num_rows: 3
})

In [59]:
train_dataset1[0]

{'chosen': "<|im_end|>\n<|im_start|>user\nUse the pygame library to write a version of the classic game Snake, with a unique twist<|im_end|>\n<|im_start|>assistant\nSure, I'd be happy to help you write a version of the classic game Snake using the pygame library! Here's a basic outline of how we can approach this:\n\n1. First, we'll need to set up the game display and create a game object that we can use to handle the game's state.\n2. Next, we'll create the game's grid, which will be used to represent the game board. We'll need to define the size of the grid and the spaces within it.\n3. After that, we'll create the snake object, which will be used to represent the player's movement. We'll need to define the size of the snake and the speed at which it moves.\n4. We'll also need to create a food object, which will be used to represent the food that the player must collect to score points. We'll need to define the location of the food and the speed at which it moves.\n5. Once we have th